In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [6]:
dataset = pd.read_csv('/Users/harshitgupta/Desktop/vs /VS-Data-Den/rm_dataset_filtered.csv')


In [7]:
dataset['date_field'] = pd.to_datetime(dataset['date_field'])

In [21]:
dataset['date_field_epoch'] = dataset['date_field'].astype(np.int64) // 10**9

In [37]:
X = dataset[['regional_master', 'date_field_epoch']].values
y = dataset['sales'].values


In [46]:
y.dtype

dtype('float64')

In [39]:
regional_masters = dataset['regional_master'].unique()

In [38]:
train_data = []
test_data = []

In [47]:
for regional_master in dataset['regional_master'].unique():
    # Filter data for the current regional_master
    data_subset = dataset[dataset['regional_master'] == regional_master]
    
    # Sort data by date_field
    data_subset = data_subset.sort_values(by='date_field_epoch')
    
    # Split data into features and target
    X = data_subset[['regional_master', 'date_field_epoch']].values
    y = data_subset['sales'].values
    
    # Perform train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    train_data.append((X_train, y_train))
    test_data.append((X_test, y_test))


In [48]:

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)


In [49]:
config = TimeSeriesTransformerConfig(
    prediction_length=12,  # Example prediction length
    distribution_output="student_t",  # Example distribution output
    loss="nll",
    encoder_layers=2,
    decoder_layers=2,
    d_model=64,
    dropout=0.1,
    attention_dropout=0.1,
    activation_dropout=0.1,
    activation_function="gelu",
    num_parallel_samples=100,
    use_cache=True
)

In [50]:
model = TimeSeriesTransformerModel(config)

In [51]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()

In [52]:
X_train.dtype

dtype('float32')

In [53]:
epochs = 10
batch_size = 32
for epoch in range(epochs):
    for X_train, y_train in train_data:
        for i in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            batch_X = torch.tensor(X_train[i:i+batch_size], dtype=torch.float32)
            batch_y = torch.tensor(y_train[i:i+batch_size], dtype=torch.float32)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()


TypeError: forward() missing 2 required positional arguments: 'past_time_features' and 'past_observed_mask'

In [54]:
epochs = 10
batch_size = 32
for epoch in range(epochs):
    for X_train, y_train in train_data:
        for i in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            batch_X = torch.tensor(X_train[i:i+batch_size], dtype=torch.float32)
            batch_y = torch.tensor(y_train[i:i+batch_size], dtype=torch.float32)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

TypeError: forward() missing 2 required positional arguments: 'past_time_features' and 'past_observed_mask'